In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import json
import glob
import os

from epilepsypcm.utils.outcome_params import seizure_onset_zone, engel_score, early_propogation, irritative_zone
from sklearn.metrics import auc
from epilepsypcm.models.base_models import *

# INPUT
# patient = string format, patient number
# paths = path to CCEP response files, in os format
# OUTPUT
# df = dataframe for one patient with
#       X features with columns: chNames, significant, n1, n2, p2 z scores,
#       n1, n2, p2 latencies, and flipped
#       and associated y outcome labels
def make_df(patient, paths):
    #extracting info from each response file
    n = 0
    stimChs = []
    for i in range(len(paths)):
        chNames = []
        # load info into python dictionary
        data = json.load(open(paths[i]))
        p_name = patient

        # Get list of channel names
        for key in data["time"]: chNames.append(key)
        # loop over each channel, and extract average time series and information about the peaks

        if n < 1:
            avgResp = np.empty((len(paths), len(chNames), len(data['time'][chNames[0]])))
            significant = np.empty((len(paths), len(chNames)))
            n1Zscore = np.empty((len(paths), len(chNames)))
            n2Zscore = np.empty((len(paths), len(chNames)))
            p2Zscore = np.empty((len(paths), len(chNames)))
            n1Latency = np.empty((len(paths), len(chNames)))
            n2Latency = np.empty((len(paths), len(chNames)))
            p2Latency = np.empty((len(paths), len(chNames)))
            flipped = np.empty((len(paths), len(chNames)))
            n += 1
            samplingRate = np.empty((len(paths)))
            window = np.empty((len(paths), 2))

        for j in range(len(chNames)):
            avgResp[i][j] = data['time'][chNames[j]]
            significant[i][j] = data['significant'][chNames[j]]
            n1Zscore[i][j] = data['zscores'][chNames[j]]['n1'][1]
            n2Zscore[i][j] = data['zscores'][chNames[j]]['n2'][1]
            p2Zscore[i][j] = data['zscores'][chNames[j]]['p2'][1]
            n1Latency[i][j] = data['zscores'][chNames[j]]['n1'][0] + data['window'][0] * data["samplingRate"] / 1000
            n2Latency[i][j] = data['zscores'][chNames[j]]['n2'][0] + data['window'][0] * data["samplingRate"] / 1000
            p2Latency[i][j] = data['zscores'][chNames[j]]['p2'][0] + data['window'][0] * data["samplingRate"] / 1000
            flipped[i][j] = data['zscores'][chNames[j]]['flipped']
        samplingRate[i] = data["samplingRate"]
        window[i] = data['window']
        stimChs = stimChs + [paths[i].split("_")[1] + "_" + paths[i].split("_")[2]]*len(chNames)


    # creating dataframe

    df = pd.DataFrame()
    df["stimChs"] = stimChs
    df["respChs"] = chNames * len(paths)
    df["significant"] = significant.flatten()
    df["n1Zscore"] = n1Zscore.flatten()
    df["n2Zscore"] = n2Zscore.flatten()
    df["p2Zscore"] = p2Zscore.flatten()
    df["n1Latency"] = n1Latency.flatten()
    df["n2Latency"] = n2Latency.flatten()
    df["p2Latency"] = p2Latency.flatten()
    df["flipped"] = flipped.flatten()
    df["patient"] = p_name

    # Dropped rows for stimulating channels since they only
    # contain stimulating waveforms / artifacts / saturated signals
    # Also zero out rows with latency values of -999.0

    # drop rows in the dataframe with latency values of -999.0
    df = df.drop(df.loc[df["n1Latency"] == -999.0].index)
    df = df.drop(df.loc[df["n1Latency"] == -499.0].index)

    # adding dataframe outcome values (1 if in SOZ, 0 if not)

    
    # adding dataframe outcome values (1 if in SOZ, 0 if not)
    df["outcome"] = np.zeros(df.shape[0])
    df["IZ"] = np.zeros(df.shape[0])
    df["EP"] = np.zeros(df.shape[0])
    
    if engel_score[patient] == "1":
        if seizure_onset_zone[patient] != ["None"]:
            for node in seizure_onset_zone[patient]:
                for channel in df["respChs"]:
                    channel_split = channel.split("_")
                    if (node == channel_split[0]) | (node == channel_split[1]):
                        df.loc[df['respChs']==channel, ['outcome']] = 1
                    elif ("0" in channel_split[0][1:-1]) | ("0" in channel_split[1][1:-1]): # LA01 vs LA1
                        if "0" in channel_split[0][1:-1]:
                            channel_new = channel_split[0].replace("0", "")
                            if node == channel_new:
                                df.loc[df['respChs']==channel, ['outcome']] = 1
                        if "0" in channel_split[1][1:-1]:
                            channel_new = channel_split[1].replace("0", "")
                            if node == channel_new:
                                df.loc[df['respChs']==channel, ['outcome']] = 1

        if irritative_zone[patient] != ["None"]:
            for IZnode in irritative_zone[patient]:
                for channel in df["respChs"]:
                    channel_split = channel.split("_")
                    if (IZnode == channel_split[0]) | (IZnode == channel_split[1]):
                        df.loc[df['respChs']==channel, ['IZ']] = 1
        if early_propogation[patient] != ["None"]:
            for EPnode in early_propogation[patient]:
                for channel in df["respChs"]:
                    channel_split = channel.split("_")
                    if (EPnode == channel_split[0]) | (EPnode == channel_split[1]):
                        df.loc[df['respChs']==channel, ['EP']] = 1
                
    return df

# Function that takes in the location of all patient folders and engel
# score of interest, and returns a nested list of dataframes for each patient
# INPUT:
# base_path = string, file location to base folder that contains all patient folders
# engel_score = string, target engel score to get dataframe for (ex. "1")
#               can currently only handle "1" and "2"
# OUTPUT:
# positive_dataframes = a nested list, where [patient number (string), dataframe].


ModuleNotFoundError: No module named 'epilepsypcm'

In [ ]:
def df_processing(D):
    D.reset_index(drop = True, inplace=True)

    #Find channel names that exists both in stimChs and respChs - only account channels that have arrows going out and in
    overlap = []
    for channel in D.respChs.unique():
        if channel in D.stimChs.unique():
            overlap.append(channel)

    #Keep only the response that were stimulated in responded in the channel in overlap list
    dropindxs = []
    for i in range(len(D)):
        if D.iloc[i].stimChs not in overlap or D.iloc[i].respChs not in overlap:
                dropindxs.append(i)
    D.drop(dropindxs,inplace=True)
    D.reset_index(drop = True, inplace=True)

    D.n1Zscore = abs(D.n1Zscore)
    D.n2Zscore = abs(D.n2Zscore)
    D.p2Zscore = abs(D.p2Zscore)
    
    #start processing
    df = pd.DataFrame()
    ChNames = overlap
    Outcomes = np.array([])
    IZ = np.array([])
    EP = np.array([])
    Per_Significant_Resp = np.array([])
    Per_Significant_Stim = np.array([])
    N1_Avg_Resp = np.array([])
    N1_STV_Resp = np.array([])
    N2_Avg_Resp = np.array([])
    N2_STV_Resp = np.array([])
    P2_Avg_Resp = np.array([])
    P2_STV_Resp = np.array([])
    N1_Avg_Stim = np.array([])
    N1_STV_Stim = np.array([])
    N2_Avg_Stim = np.array([])
    N2_STV_Stim = np.array([])
    P2_Avg_Stim = np.array([])
    P2_STV_Stim = np.array([])

    for channel in ChNames:
        Resp = D[D.respChs == channel]
        Stim = D[D.stimChs == channel]

        Outcomes = np.append(Outcomes,Resp[:1].outcome)
        IZ = np.append(IZ,Resp[:1].IZ)
        EP = np.append(EP,Resp[:1].EP)

        Per_Significant_Resp = np.append(Per_Significant_Resp,
                                         sum(Resp.significant/len(Resp)))
        Per_Significant_Stim = np.append(Per_Significant_Stim,
                                         sum(Stim.significant/len(Stim)))

        N1_Avg_Resp = np.append(N1_Avg_Resp,sum(Resp.n1Zscore)/len(Resp))
        N1_STV_Resp = np.append(N1_STV_Resp,np.std(Resp.n1Zscore))

        N2_Avg_Resp = np.append(N2_Avg_Resp,sum(Resp.n2Zscore)/len(Resp))
        N2_STV_Resp = np.append(N2_STV_Resp,np.std(Resp.n2Zscore))

        P2_Avg_Resp = np.append(P2_Avg_Resp,sum(Resp.p2Zscore)/len(Resp))
        P2_STV_Resp = np.append(P2_STV_Resp,np.std(Resp.p2Zscore))

        N1_Avg_Stim = np.append(N1_Avg_Stim,sum(Stim.n1Zscore)/len(Stim))
        N1_STV_Stim = np.append(N1_STV_Stim,np.std(Stim.n1Zscore))

        N2_Avg_Stim = np.append(N2_Avg_Stim,sum(Stim.n2Zscore)/len(Stim))
        N2_STV_Stim = np.append(N2_STV_Stim,np.std(Stim.n2Zscore))

        P2_Avg_Stim = np.append(P2_Avg_Stim,sum(Stim.p2Zscore)/len(Stim))
        P2_STV_Stim = np.append(P2_STV_Stim,np.std(Stim.p2Zscore))


    df['Channels'] = ChNames
    df['outcome'] = Outcomes
    df['IZ'] = IZ
    df['EP'] = EP
    df['SigResp'] = Per_Significant_Resp
    df['SigStim'] = Per_Significant_Stim
    df['N1RespAvg'] = N1_Avg_Resp
    df['N1RespSDV'] = N1_STV_Resp
    df['N2RespAvg'] = N2_Avg_Resp
    df['N2RespSDV'] = N2_STV_Resp
    df['P2RespAvg'] = P2_Avg_Resp
    df['P2RespSDV'] = P2_STV_Resp
    df['N1StimAvg'] = N1_Avg_Stim
    df['N1StimSDV'] = N1_STV_Stim
    df['N2StimAvg'] = N2_Avg_Stim
    df['N2StimSDV'] = N2_STV_Stim
    df['P2StimAvg'] = P2_Avg_Stim
    df['P2StimSDV'] = P2_STV_Stim
    df['patient'] = D.iloc[0].patient
    
    df["InDegree"] = np.zeros(df.shape[0])
    df["OutDegree"] = np.zeros(df.shape[0])
    df["EV"] = np.zeros(df.shape[0])
    df["Closeness"] = np.zeros(df.shape[0])
    
    G = nx.DiGraph()
    for i in range(D.shape[0]):
        if D.significant.iloc[i] == 1:
            G.add_edge(D.stimChs.iloc[i],D.respChs.iloc[i])

    EV_Centrality = nx.eigenvector_centrality(G)
    Closeness_Centrality = nx.closeness_centrality(G)
    InDegree = nx.in_degree_centrality(G)
    OutDegree = nx.out_degree_centrality(G)
    
    for channel in list(EV_Centrality):
        df.loc[df.Channels == channel, 'EV'] = EV_Centrality[channel]
    for channel in list(Closeness_Centrality):
        df.loc[df.Channels == channel, 'Closeness'] = Closeness_Centrality[channel]
    for channel in list(InDegree):
        df.loc[df.Channels == channel, 'InDegree'] = InDegree[channel]
    for channel in list(OutDegree):
        df.loc[df.Channels == channel, 'OutDegree'] = OutDegree[channel]

            
    return df

In [ ]:
import glob
import os
from pathlib import Path

def get_df_list(base_path, engel):
    patient_files = os.listdir(base_path)

    positive_dataframes = []
    for file in patient_files:
        if (file[0] == "P") & (file != "PY16N006"):
            response_path = base_path + file + '/ResponseInfo/CCEP'
            response_files_path = glob.glob(response_path + '/*.json', recursive=True)

            # Getting individual dataframe for positive patients
            patient = file
            if file in engel_score.keys():  # if we currently have the file's engel score
                if engel_score[patient] == engel:  # if the engel score is 1
                    df = make_df(patient, response_files_path)
                    positive_dataframes.append([patient, df])

    return positive_dataframes

# Function that combines dataframes for all patients of a particular
# engel class
# INPUT:
# base_path = string, file location to base folder that contains all patient folders
# engel_score = string, target engel score to get dataframe for (ex. "1")
#               can currently only handle "1" and "2"
# balance (OPTIONAL, default = None) = "None", "upsample", or "downsample"
#          will upsample minority class or downsample majority class to balance
#           the data
# OUTPUT:
# all_positive_patients = a concatonated dataframe of all patients


In [ ]:

from sklearn.utils import resample

def concat_dfs(base_path, engel, balance = None):
    
    patient_files = os.listdir(base_path)

    full_df = pd.DataFrame()
    for file in patient_files:
        if (file[0] == "P") & (file != "PY16N006") & (file != 'PY17N014'): #PY17N014 was eliminated because there is no node with significant response
            response_path = base_path + file + '/ResponseInfo/CCEP'
            response_files_path = glob.glob(response_path + '/*.json', recursive=True)

            # Getting individual dataframe for positive patients
            patient = file
            if file in engel_score.keys():  # if we currently have the file's engel score
                if engel_score[patient] == engel:  # if the engel score is 1
                    df = make_df(patient, response_files_path)
                    df = df_processing(df)
                    full_df = pd.concat([full_df, df])
                    
                    print('%s done...'%patient)

    # seperate dataframes for class
    df_majority = full_df[full_df.outcome == 0]
    df_minority = full_df[full_df.outcome == 1]

    # upsample data if balance parameter is set to "Upsample" or "upsample"
    if (balance == "upsample") | (balance == "Upsample"):
        # Upsample minority class
        df_minority_upsampled = resample(df_minority,
                                         replace=True,  # sample with replacement
                                         n_samples=full_df["outcome"].value_counts()[0.0],
                                         # to match majority class
                                         random_state=123)  # reproducible results


        # combine dataframes
        full_df = pd.concat([df_majority, df_minority_upsampled])

    # downsample data if balance parameter is set to "downsample" or "Downsample"
    elif (balance == "downsample") | (balance == "Downsample"):
        # downsample majority class
        # downsample majority class
        df_majority_downsampled = resample(df_majority,
                                           replace=False,  # sample without replacement
                                           n_samples= full_df["outcome"].value_counts()[1.0],
                                           # to match minority class
                                           random_state=123)  # reproducible results


        full_df = pd.concat([df_majority_downsampled, df_minority])

    return full_df


# Function that upsamples or downsamples a training set to balance classes
# INPUT:
# X_train = output from train_test_split function
# y_train = output from train_test_split function
# balance = "upsample", or "downsample"
#          will upsample minority class or downsample majority class to balance
#           the data
# OUTPUT:
# X_train = new balanced X training data
# y_train = new balanced y training data

In [ ]:
from sklearn.utils import resample
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

def class_balance(X_train, y_train, balance):
    full_df = pd.concat([X_train, y_train], axis = 1)

    # seperate dataframes for class
    df_majority = full_df[full_df.outcome == 0]
    df_minority = full_df[full_df.outcome == 1]

    # upsample data if balance parameter is set to "Upsample" or "upsample"
    if (balance == "upsample") | (balance == "Upsample"):
        # Upsample minority class
        df_minority_upsampled = resample(df_minority,
                                        replace=True,  # sample with replacement
                                        n_samples=full_df["outcome"].value_counts()[0.0],
                                        # to match majority class
                                        random_state=123)  # reproducible results


        # combine dataframes
        full_df = pd.concat([df_majority, df_minority_upsampled])

    # downsample data if balance parameter is set to "downsample" or "Downsample"
    elif (balance == "downsample") | (balance == "Downsample"):
        # downsample majority class
        # downsample majority class
        df_majority_downsampled = resample(df_majority,
                                        replace=False,  # sample without replacement
                                        n_samples= full_df["outcome"].value_counts()[1.0],
                                        # to match minority class
                                        random_state=123)  # reproducible results


        full_df = pd.concat([df_majority_downsampled, df_minority])

    X_train = full_df.drop(columns = ["outcome"])
    y_train = full_df["outcome"]
    
    return X_train, y_train


In [ ]:
base_path = '/Users/richardlee/Desktop/JHU/2021 Fall/Precision Care Medicine/Preprocessed_Data/'



#Function to get the concatenated dataframe for all positive patients
## balance parameter can be changed to "None", "upsample", or "downsample"
all_positive_patients = concat_dfs(base_path, "1")

In [ ]:
all_positive_patients.reset_index(drop = True, inplace=True)
all_positive_patients

In [ ]:
all_positive_patients.to_csv('newDF.csv')

In [ ]:
A = all_positive_patients
A = A.Channels.unique()
B = []
for channels in A:
    electrodes = channels.split('_')
    if electrodes[0][-1].isdigit():
        electrodes[0] = electrodes[0][0:-1]
    if electrodes[0][-1].isdigit():
        electrodes[0] = electrodes[0][0:-1]
    B.append(electrodes[0])    
    
    if electrodes[1][-1].isdigit():
        electrodes[1] = electrodes[1][0:-1]
    if electrodes[1][-1].isdigit():
        electrodes[1] = electrodes[1][0:-1]
    B.append(electrodes[1]) 
B = pd.Series(B)
B.unique()

In [ ]:
A

In [ ]:
file = 'PY21N008'
patient = file
response_path = base_path + file + '/ResponseInfo/CCEP'
response_files_path = glob.glob(response_path + '/*.json', recursive=True)
df = make_df(patient, response_files_path)
df

In [ ]:
all_positive_patients[all_positive_patients.EV==0]

In [ ]:
all_positive_patients[all_positive_patients.outcome == 1]

In [ ]:
all_positive_patients[all_positive_patients.patient=='PY19N026']

In [ ]:
all_positive_patients.patient.unique()